# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [200]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [201]:
test = pd.read_csv('flights_test.csv')

In [202]:
df = pd.read_csv('flights_merged.csv', index_col=0)

In [203]:
df = df.dropna(axis=0, subset=['crs_elapsed_time'])

In [204]:
df = df.sample(frac=0.1)

In [205]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,passengers,capacity,total_gallons,dep_delay,nas_delay,security_delay,late_aircraft_delay,weather_delay,carrier_delay
729986,2019-01-25,B6,B6,B6,205,B6,N589JB,205,11618,EWR,...,1065.0,343.600000,26.400000,7.403417e+07,49.988345,12.564103,0.044289,24.757576,0.363636,19.125874
213756,2019-11-15,AA,AA,AA,190,AA,N9010R,190,11298,DFW,...,925.0,1867.666667,703.666667,3.079852e+08,55.833333,5.661111,0.000000,24.738889,2.333333,22.994444
627503,2019-04-04,WN,WN,WN,767,WN,N226WN,767,14869,SLC,...,391.0,1375.333333,364.000000,1.723195e+08,47.058020,14.129693,0.000000,20.682594,0.423208,11.894198
377680,2019-04-26,AS,AS,AS,524,AS,N492AS,524,15304,TPA,...,2520.0,5742.666667,487.333333,6.101888e+07,38.283333,16.016667,0.000000,19.833333,2.950000,8.866667
530669,2019-06-10,UA,UA,UA,2163,UA,N37290,2163,14679,SAN,...,2425.0,8807.000000,821.000000,2.975601e+08,54.469274,38.145251,0.000000,14.972067,0.000000,5.944134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392402,2019-11-09,DL,DL,DL,2128,DL,N587NW,2128,11433,DTW,...,957.0,24361.000000,3950.000000,3.087060e+08,40.737828,15.479401,0.000000,9.580524,2.625468,23.756554
19030,2019-05-01,NK,NK,NK,841,NK,N625NK,841,14524,RIC,...,667.0,3810.666667,721.000000,3.937194e+07,33.500000,16.676471,0.000000,17.882353,0.000000,8.823529
543300,2019-12-18,WN,WN,WN,632,WN,N239WN,632,10693,BNA,...,471.0,8381.750000,2083.250000,1.723195e+08,40.321429,5.984694,0.500000,22.260204,1.265306,12.153061
303535,2019-03-26,AA,AA,AA,2836,AA,N563UW,2836,11057,CLT,...,544.0,160.000000,0.000000,3.079852e+08,46.669797,23.748044,0.173709,17.161189,0.957746,11.226917


In [206]:
y= df['arr_delay']

In [207]:
X = df

In [208]:
X = X.drop(columns=['arr_delay'])

In [227]:
X.fl_date = X.fl_date.apply(pd.to_datetime)

AttributeError: 'DataFrame' object has no attribute 'fl_date'

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [210]:
X = X[['origin_airport_id', 'dest_airport_id', 'distance', 'passengers', 'crs_elapsed_time','capacity', 'total_gallons', 'dep_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay',
       'weather_delay', 'carrier_delay']]
#dropped flight date :( 

In [211]:
x_label = X.columns

In [212]:
scaler = StandardScaler()

In [213]:
X = pd.DataFrame(scaler.fit_transform(X))

In [214]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8)


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [215]:
#data_dmatrix = xgb.DMatrix(data=X,label=y, enable_categorical=True)

In [216]:
#objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 7, alpha = 0.1,

In [217]:
lr = LinearRegression()

In [218]:
lr.fit(X_train, y_train) 

LinearRegression()

In [219]:
lr.score(X_train, y_train)

0.03475286398060684

In [220]:
lr.score(X_test, y_test)

0.033550358543344094

### Random Forest

RandomForestRegressor()

In [221]:
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100)
 

In [222]:
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

RandomForestRegressor()

In [223]:
y_pred = regressor.predict(X_test)

In [224]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 33.84939850414698
Mean Squared Error: 2053.7701070852418
Root Mean Squared Error: 45.3185404341892


In [225]:
feature_importance= pd.Series(regressor.feature_importances_, index= x_label).sort_values(ascending=False)
feature_importance

crs_elapsed_time       0.582456
dep_delay              0.073987
total_gallons          0.048307
nas_delay              0.041661
carrier_delay          0.033204
weather_delay          0.032092
late_aircraft_delay    0.031892
capacity               0.030632
passengers             0.029467
distance               0.029427
origin_airport_id      0.027557
dest_airport_id        0.026043
security_delay         0.013274
dtype: float64

In [172]:
params = {'n_estimators' : [100, 150, 200, 300], 'max_depth': [3, 10, 20, 30]}

In [173]:
grid = GridSearchCV(regressor, params)

In [ ]:
grid_result = grid.fit(X_train, y_train)

In [ ]:
print(grid_result.best_score_)
print( grid_result.best_params_['n_estimators'])
print( grid_result.best_params_['max_depth'])

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.